In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Patient_Data.csv")

In [9]:
print(df)

   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
1        102      Bob    Neurology   Dr. John         NaN               2   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
3        104    David   Cardiology  Dr. Smith      6200.0               3   
4        105      Eva  Dermatology   Dr. Rose         NaN               2   
5        101    Alice   Cardiology  Dr. Smith      5000.0               1   

        CheckInTime  
0  2023-01-10 09:00  
1  2023-01-11 10:30  
2  2023-01-12 11:00  
3  2023-01-13 12:00  
4  2023-01-14 08:45  
5  2023-01-10 09:00  


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None


In [6]:
billing_df = pd.read_csv("Billing_Data.csv")

In [8]:
print(billing_df)

   PatientID  InsuranceCovered  FinalAmount
0        101              2000         3000
1        102              1500         3500
2        103              2500         5000
3        104              3000         3200
4        105              1000         4000


In [13]:
billing_cols = df[['PatientID', 'Department', 'Doctor', 'BillAmount']]
print(billing_cols)


   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN
5        101   Cardiology  Dr. Smith      5000.0


In [14]:
cols_to_drop = ['ReceptionistID', 'CheckInTime']
df_clean = df.drop(columns=[c for c in cols_to_drop if c in df.columns]).copy()

In [15]:
print(df_clean)

   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN
5        101    Alice   Cardiology  Dr. Smith      5000.0


In [16]:
df_clean = df_clean.drop_duplicates(subset='PatientID', keep='first').reset_index(drop=True)

In [17]:
print(df_clean)

   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


In [18]:
if 'BillAmount' in df_clean.columns:
    mean_bill = df_clean['BillAmount'].mean(skipna=True)
    df_clean['BillAmount'] = df_clean['BillAmount'].fillna(mean_bill)
else:
    mean_bill = None
    print("Warning: 'BillAmount' not found in df_clean")

In [19]:
dept_totals = df_clean.groupby('Department', as_index=False)['BillAmount'].sum().rename(columns={'BillAmount':'TotalBillAmount'})
print("Total bill amount per department:\n", dept_totals)


Total bill amount per department:
     Department  TotalBillAmount
0   Cardiology     11200.000000
1  Dermatology      6233.333333
2    Neurology      6233.333333
3  Orthopedics      7500.000000


In [20]:
if 'PatientID' in billing_df.columns:
    billing_df_clean = billing_df.drop_duplicates(subset='PatientID', keep='first').reset_index(drop=True)
    merged = pd.merge(df_clean, billing_df_clean, on='PatientID', how='left')
else:
    raise KeyError("billing_df must contain 'PatientID' for merging")

print("\nMerged dataframe (first rows):\n", merged.head())


Merged dataframe (first rows):
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  


In [21]:
new_patients = pd.DataFrame([
    {'PatientID': 106, 'Name': 'Frank', 'Department': 'ENT', 'Doctor': 'Dr. Green', 'BillAmount': 4200},
    {'PatientID': 107, 'Name': 'Grace', 'Department': 'Cardiology', 'Doctor': 'Dr. Smith', 'BillAmount': np.nan},  # will fill later
])

In [22]:
print(new_patients)

   PatientID   Name  Department     Doctor  BillAmount
0        106  Frank         ENT  Dr. Green      4200.0
1        107  Grace  Cardiology  Dr. Smith         NaN


In [23]:
for col in merged.columns:
    if col not in new_patients.columns:
        new_patients[col] = np.nan
# Reorder columns to match merged
new_patients = new_patients[merged.columns]


In [24]:
print(new_patients)

   PatientID   Name  Department     Doctor  BillAmount  InsuranceCovered  \
0        106  Frank         ENT  Dr. Green      4200.0               NaN   
1        107  Grace  Cardiology  Dr. Smith         NaN               NaN   

   FinalAmount  
0          NaN  
1          NaN  


In [25]:
if mean_bill is not None and 'BillAmount' in new_patients.columns:
    new_patients['BillAmount'] = new_patients['BillAmount'].fillna(mean_bill)

concatenated_rows = pd.concat([merged, new_patients], ignore_index=True)
print("\nAfter adding new patients (rows):\n", concatenated_rows.tail())



After adding new patients (rows):
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   
5        106    Frank          ENT  Dr. Green  4200.000000               NaN   
6        107    Grace   Cardiology  Dr. Smith  6233.333333               NaN   

   FinalAmount  
2       5000.0  
3       3200.0  
4       4000.0  
5          NaN  
6          NaN  


In [27]:
if {'InsuranceCovered','FinalAmount'}.issubset(billing_df_clean.columns):
    final_df = pd.merge(concatenated_rows, billing_df_clean[['PatientID','InsuranceCovered','FinalAmount']], on='PatientID', how='left', suffixes=('','_billing'))
else:
    # Option B: If you have these columns as a separate DataFrame that aligns row-wise with concatenated_rows:
    new_billing_cols = pd.DataFrame({
        'InsuranceCovered': [2000]*len(concatenated_rows),   # placeholder values
        'FinalAmount': concatenated_rows['BillAmount'] + 1000  # example derivation
    })
    
    if len(new_billing_cols) == len(concatenated_rows):
        final_df = pd.concat([concatenated_rows.reset_index(drop=True), new_billing_cols.reset_index(drop=True)], axis=1)
    else:
        if 'PatientID' in billing_df_clean.columns:
            final_df = pd.merge(concatenated_rows, billing_df_clean[['PatientID','InsuranceCovered','FinalAmount']], on='PatientID', how='left')
        else:
            raise ValueError("Cannot safely concatenate column-wise: lengths mismatch and no PatientID to merge on.")

print("\nFinal dataframe (first rows):\n", final_df.head())


Final dataframe (first rows):
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000            2000.0   
1        102      Bob    Neurology   Dr. John  6233.333333            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   

   FinalAmount  InsuranceCovered_billing  FinalAmount_billing  
0       3000.0                    2000.0               3000.0  
1       3500.0                    1500.0               3500.0  
2       5000.0                    2500.0               5000.0  
3       3200.0                    3000.0               3200.0  
4       4000.0                    1000.0               4000.0  
